# LSTM

In [40]:
import torch
from torch import nn, optim
from torchtext import data, datasets

print("GPU:", torch.backends.mps.is_available())
device = torch.device("mps")

torch.manual_seed(123)


GPU: True


In [52]:
from torchtext.data import Field, LabelField

ImportError: cannot import name 'TabularDataset' from 'torchtext.datasets' (/Users/shuowang/anaconda3/envs/pytorch/lib/python3.9/site-packages/torchtext/datasets/__init__.py)

In [42]:
# TEXT = data.Field(tokenize="spacy")
# LABEL = data.LabelField(dtype=torch.float)
# train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

# print("len of train data:", len(train_data))
# print("len of test data:", len(test_data))

# print(train_data.examples[15].text)
# print(train_data.examples[15].label)

# # word2vec, glove
# TEXT.build_vocab(train_data, max_size=10000, vectors="glove.6B.100d")
# LABEL.build_vocab(train_data)


In [51]:
import spacy
import torch
from torchtext.datasets import IMDB
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import Vocab
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Subset

nlp = spacy.load("en_core_web_sm")
tokenizer = get_tokenizer(lambda s: [token.text for token in nlp.tokenizer(s)])


def collate_batch(batch):
    labels = torch.tensor([example.label for example in batch])
    text = [tokenizer(example.text) for example in batch]
    text_lengths = torch.tensor([len(t) for t in text])
    text = [torch.tensor([vocab.stoi[token] for token in t]) for t in text]
    text = torch.nn.utils.rnn.pad_sequence(text, batch_first=True)
    return text, text_lengths, labels


train_data, test_data = IMDB(split=("train", "test"))

tokens = []
for example in train_data:
    tokens += tokenizer(example.text)

vocab = Vocab(
    freqs=None,
    min_freq=1,
    specials=["<unk>", "<pad>"],
    unk_token="<unk>",
    pad_token="<pad>",
)
vocab.set_default_index()

for token in tokens:
    vocab.freqs[token] += 1

train_loader = DataLoader(
    train_data, batch_size=32, shuffle=True, collate_fn=collate_batch
)


AttributeError: 'NoneType' object has no attribute 'Lock'
This exception is thrown by __iter__ of _MemoryCellIterDataPipe(remember_elements=1000, source_datapipe=_ChildDataPipe)